In [78]:
import pandas as pd
import numpy as np
import lightgbm as lgb
#from sklearn.linear_model import LinearRegression as reg
from statsmodels.regression.linear_model import OLS
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import confusion_matrix, roc_auc_score


pd.set_option('display.max_columns', 50)

In [2]:
def metrics(y_true, y_pred):
    '''
    Calculates binary classification performance metrics for a given model.
    :param y_true: array_like, truth values as int
    :param y_pred: array_like, predicted values as int
    :returns: dict, with keys for each metric: 
        accuracy - proportion of correct predictions out of total predictions
        sensitivity - (aka recall), of all true positives reviews how many did we correctly predict as positive
        specificity - (aka selectivity/TNR), of all true negatives how many did we correctly predict as negative
        precision - of all predicted positive cases how many were actually positive
        F-1 score - harmonic/weighted mean of precision and sensitivity scores
        ROC-AUC - area under receiver operating characteristic curve
        
    '''
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    
    metrics_dict = {}
    metrics_dict['accuracy'] = round((tp + tn) / len(y_true), 4)
    metrics_dict['sensitivity/recall'] = round(tp / (fn + tp), 4) # aka recall
    metrics_dict['specificity'] = round(tn / (tn + fp), 4) # aka TNR
    metrics_dict['precision'] = round(tp / (tp + fp), 4)
    metrics_dict['f1'] = round(2 * (metrics_dict['precision'] * metrics_dict['sensitivity/recall']) \
                        / (metrics_dict['precision'] + metrics_dict['sensitivity/recall']), 4)
    metrics_dict['roc_auc'] = round(roc_auc_score(y_true, y_pred), 4)
    
    return metrics_dict

In [56]:
def combine_prediction(X_test, pred, outcome='50k'):
    X_test_combined = pd.concat([X_test, pd.DataFrame(y_test, columns=[outcome])], axis=1)
    return pd.concat([X_test_combined.reset_index(drop=True), pd.DataFrame(pred, columns=['pred'])], axis=1)

# Data preparation

### Read in [2019 ASEC (March CPS)](https://cps.ipums.org/cps/)

In [3]:
df = pd.read_csv('./data/asec_2019.csv')

In [4]:
print(df.shape)

(180101, 18)


In [5]:
print(df.head())

   YEAR  MONTH  ASECFLAG   ASECWT  RELATE  AGE  SEX  RACE  MARST  POPSTAT  \
0  2019      3         1  2031.67     101   21    1   100      6        1   
1  2019      3         1  1232.04     101   85    2   100      5        1   
2  2019      3         1  1209.17     101   61    2   100      6        1   
3  2019      3         1  1146.23     101   73    2   100      4        1   
4  2019      3         1  1480.79     301   37    1   100      6        1   

   HISPAN  LABFORCE  UHRSWORK1  EDUC  SCHLCOLL  WKSWORK1  UHRSWORKLY  INCWAGE  
0       0         2       30.0    60       5.0        52          30  18000.0  
1       0         1      999.0    73       0.0         0         999      0.0  
2       0         2       44.0    73       0.0        52          44  12000.0  
3       0         1      999.0    73       0.0         0         999      0.0  
4       0         2       20.0    73       5.0        52          20  12000.0  


### Restrict dataset

In [6]:
# Drop individuals with no employment or earnings last year
df = df.loc[df['INCWAGE'] > 0] # No wage/salary last year
df = df.loc[(df['WKSWORK1'] > 0) & (df['WKSWORK1'] <= 52)] # No weeks worked last year
df = df.loc[(df['UHRSWORKLY'] > 0) & (df['UHRSWORKLY'] < 999)] # No usual hours/week worked last year

In [7]:
print(df.shape)

(85644, 18)


In [8]:
# Restrict to individuals aged 18-64
df = df.loc[(df['AGE'] >=18) & (df['AGE'] <=64)]

In [9]:
print(df.shape)

(78644, 18)


In [10]:
# Restrict to non-Hispanics
df = df.loc[df['HISPAN'] == 0]

In [11]:
print(df.shape)

(63125, 18)


In [12]:
# Restrict to non-mixed-race Blacks and Whites
df = df.loc[df.RACE.isin([100, 200])]
df['blk'] = np.where(df['RACE'] == 200, 1, 0)

In [13]:
print(df.shape)

(55508, 19)


In [14]:
# Restrict to adult civilians
df = df.loc[df['POPSTAT'] == 1]

In [15]:
print(df.shape)

(55069, 19)


### Engineer features

In [16]:
# Flag for part-time usual work
df['pt'] = np.where(df['UHRSWORKLY'] < 35, 1, 0)

In [17]:
# Non-linearities for age
df['AGE2'] = df['AGE'] ** 2
df['AGE3'] = df['AGE'] ** 3

In [18]:
# Ensure education coded correctly
df = df.loc[df['EDUC'] <= 125] # 999 is missing

In [19]:
# Hourly wage
df['hrwage'] = df['INCWAGE'] / (df['WKSWORK1'] * df['UHRSWORKLY'])

In [20]:
df['hrwage'].describe()

count    55069.000000
mean        31.269112
std        184.416973
min          0.000962
25%         14.375000
50%         21.634615
75%         34.188034
max      25481.250000
Name: hrwage, dtype: float64

In [21]:
# Restrict to people earning 1 < hr_wage < 100
df = df.loc[(df['hrwage'] > 1) & (df['hrwage'] < 100)]

In [22]:
print(df['hrwage'].describe())

count    53671.000000
mean        25.907302
std         17.049376
min          1.016667
25%         14.202864
50%         21.634615
75%         33.333333
max         99.759615
Name: hrwage, dtype: float64


In [23]:
# Log hourly wage
df['lnwage'] = np.log(df['hrwage'])

In [24]:
df['INCWAGE'].describe()

count     53671.000000
mean      52682.839876
std       41242.872191
min          25.000000
25%       24617.500000
50%       43680.000000
75%       70000.000000
max      420000.000000
Name: INCWAGE, dtype: float64

In [25]:
# Flag for whether total earnings > 50,000 or not
df['50k'] = np.where(df['INCWAGE'] > 50000, 1, 0)

In [26]:
df = df[['lnwage', 'hrwage', '50k', 'pt', 'INCWAGE', 'WKSWORK1', 'UHRSWORKLY', 
         'AGE', 'AGE2', 'AGE3', 'SEX', 'blk', 'MARST', 'SCHLCOLL', 'EDUC']]

In [27]:
print(df.head())

     lnwage     hrwage  50k  pt  INCWAGE  WKSWORK1  UHRSWORKLY  AGE  AGE2  \
0  2.445686  11.538462    0   1  18000.0        52          30   21   441   
2  1.657229   5.244755    0   0  12000.0        52          44   61  3721   
4  2.445686  11.538462    0   1  12000.0        52          20   37  1369   
6  3.179655  24.038462    1   0  55000.0        52          44   53  2809   
8  3.442059  31.251250    1   0  50002.0        40          40   62  3844   

     AGE3  SEX  blk  MARST  SCHLCOLL  EDUC  
0    9261    1    0      6       5.0    60  
2  226981    2    0      6       0.0    73  
4   50653    1    0      6       5.0    73  
6  148877    2    0      4       5.0    73  
8  238328    1    0      1       0.0   111  


In [28]:
categorical_features = ['SEX', 'MARST', 'SCHLCOLL', 'EDUC', 'pt', 'blk']
for col in categorical_features:
    df[col] = df[col].astype('category')

In [29]:
df.dtypes

lnwage         float64
hrwage         float64
50k              int32
pt            category
INCWAGE        float64
WKSWORK1         int64
UHRSWORKLY       int64
AGE              int64
AGE2             int64
AGE3             int64
SEX           category
blk           category
MARST         category
SCHLCOLL      category
EDUC          category
dtype: object

# Raw difference in earnings by race 

### Association between race and likelihood of annual earnings surpassing 50K (i.e. "high-income")

In [30]:
np.corrcoef(df['50k'], df['blk'])

array([[ 1.        , -0.10890816],
       [-0.10890816,  1.        ]])

As compared to being white, being black is associatd with a lower likelihood of being high-income.

In [31]:
print(f"Percent of high-income blacks: {round(df[df.blk==1]['50k'].mean()*100, 2)}%")
print(f"Percent of high-income whites: {round(df[df.blk==0]['50k'].mean()*100, 2)}%")
print(f"Difference: {round((df[df.blk==0]['50k'].mean() - df[df.blk==1]['50k'].mean())*100, 2)}%")

Percent of high-income blacks: 27.9%
Percent of high-income whites: 42.76%
Difference: 14.86%


### Association between race and log hourly wage

In [32]:
np.corrcoef(df['lnwage'], df['blk'])

array([[ 1.        , -0.11854933],
       [-0.11854933,  1.        ]])

Being black is also correlated with lower hourly wages

In [36]:
print(f"Mean hourly wage, Blacks: ${round(df[df.blk==1]['hrwage'].mean(), 2)}")
print(f"Mean hourly wage, Whites: ${round(df[df.blk==0]['hrwage'].mean(), 2)}")
print(f"Difference: ${round((df[df.blk==0]['hrwage'].mean() - df[df.blk==1]['hrwage'].mean()), 2)}")

Mean hourly wage, Blacks: $21.34
Mean hourly wage, Whites: $26.73
Difference: $5.39


# Discrimination in predicting high/low earners

### Train-test split

In [38]:
y = df['50k']
X = df[['blk', 'AGE', 'AGE2', 'AGE3', 'SEX', 'EDUC', 'SCHLCOLL', 'MARST', 'pt']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=999)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=999)

In [39]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(38642, 9)
(9661, 9)
(5368, 9)


### Discriminatory model: Includes race

In [40]:
model1 = lgb.LGBMClassifier(objective='binary',
                           random_state=999,
                           metric='logloss')

In [41]:
model1.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric='logloss', early_stopping_rounds=10, verbose=False)

LGBMClassifier(metric='logloss', objective='binary', random_state=999)

In [42]:
disc_pred = model1.predict(X_test)

In [43]:
metrics(y_test, disc_pred)

{'accuracy': 0.7547,
 'sensitivity/recall': 0.7206,
 'specificity': 0.7773,
 'precision': 0.6827,
 'f1': 0.7011,
 'roc_auc': 0.749}

In [52]:
discrim = combine_prediction(X_test, disc_pred)

In [55]:
print("Model performance: Blacks")
mask = discrim[discrim.blk==1]
metrics(mask['50k'], mask['pred'])

Model performance: Blacks


{'accuracy': 0.7799,
 'sensitivity/recall': 0.5244,
 'specificity': 0.874,
 'precision': 0.6051,
 'f1': 0.5619,
 'roc_auc': 0.6992}

In [58]:
print("Model performance: Whites")
mask = discrim[discrim.blk==0]
metrics(mask['50k'], mask['pred'])

Model performance: Whites


{'accuracy': 0.75,
 'sensitivity/recall': 0.7436,
 'specificity': 0.7547,
 'precision': 0.69,
 'f1': 0.7158,
 'roc_auc': 0.7492}

In [69]:
print(f"Percent of Blacks predicted to be high-income: {round(discrim[discrim.blk==1]['pred'].mean()*100, 2)}%")
print(f"Percent of Whites predicted to be high-income: {round(discrim[discrim.blk==0]['pred'].mean()*100, 2)}%")
print(f"Difference: {round((discrim[discrim.blk==0]['50k'].mean() - discrim[discrim.blk==1]['50k'].mean())*100, 2)}%")

Percent of Blacks predicted to be high-income: 23.33%
Percent of Whites predicted to be high-income: 45.63%
Difference: 15.43%


### Naive model: Excluding race

In [60]:
model2 = lgb.LGBMClassifier(objective='binary',
                           random_state=999,
                           metric='logloss')

In [61]:
X_train_mod = X_train.loc[:, X_train.columns != 'blk']
X_val_mod = X_val.loc[:, X_val.columns != 'blk']
X_test_mod = X_test.loc[:, X_test.columns != 'blk']
model2.fit(X_train_mod, y_train, eval_set=[(X_val_mod, y_val)], eval_metric='logloss', early_stopping_rounds=10, verbose=False)

LGBMClassifier(metric='logloss', objective='binary', random_state=999)

In [62]:
naive_pred = model2.predict(X_test_mod)

In [63]:
metrics(y_test, naive_pred)

{'accuracy': 0.7493,
 'sensitivity/recall': 0.6884,
 'specificity': 0.7897,
 'precision': 0.6852,
 'f1': 0.6868,
 'roc_auc': 0.7391}

In [64]:
naive = combine_prediction(X_test, naive_pred)

In [66]:
print("Model performance: Blacks")
mask = naive[naive.blk==1]
metrics(mask['50k'], mask['pred'])

Model performance: Blacks


{'accuracy': 0.7715,
 'sensitivity/recall': 0.6756,
 'specificity': 0.8069,
 'precision': 0.563,
 'f1': 0.6142,
 'roc_auc': 0.7412}

In [67]:
print("Model performance: Whites")
mask = naive[naive.blk==0]
metrics(mask['50k'], mask['pred'])

Model performance: Whites


{'accuracy': 0.7451,
 'sensitivity/recall': 0.6899,
 'specificity': 0.7857,
 'precision': 0.7028,
 'f1': 0.6963,
 'roc_auc': 0.7378}

In [70]:
print(f"Percent of Blacks predicted to be high-income: {round(naive[naive.blk==1]['pred'].mean()*100, 2)}%")
print(f"Percent of Whites predicted to be high-income: {round(naive[naive.blk==0]['pred'].mean()*100, 2)}%")
print(f"Difference: {round((naive[naive.blk==0]['50k'].mean() - naive[naive.blk==1]['50k'].mean())*100, 2)}%")

Percent of Blacks predicted to be high-income: 32.3%
Percent of Whites predicted to be high-income: 41.57%
Difference: 15.43%


In [84]:
y = df['50k']
X_reg = df[['blk', 'AGE', 'AGE2', 'AGE3', 'SEX', 'MARST', 'pt', 'EDUC', 'SCHLCOLL']]
for x in X_reg.columns:
    X_reg[x] = X_reg[x].astype(float)
#X_reg = df[['blk', 'AGE', 'AGE2', 'AGE3', 'SEX', 'MARST', 'pt']]
results = OLS(y, X_reg,hasconst=False).fit()
print(results.params)
print(results.tvalues)

blk        -0.074327
AGE        -0.014878
AGE2        0.000765
AGE3       -0.000008
SEX        -0.181746
MARST      -0.018293
pt         -0.261424
EDUC        0.007673
SCHLCOLL   -0.003221
dtype: float64
blk        -14.458406
AGE        -13.127495
AGE2        19.673391
AGE3       -19.238039
SEX        -49.256033
MARST      -19.718031
pt         -49.979573
EDUC        82.793255
SCHLCOLL    -1.780773
dtype: float64


c:\users\jking\documents\pythonvirtualenv\ml_discrimination\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
